In [56]:
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass, asdict

SENSOR_RATES = {
    "glass_accelerometer": 5, "glass_gyroscope": 5, "glass_magnetometer": 5,
    "phone_acceleromter_calibrated": 100, "phone_accelerometer": 500,
    "phone_gravity": 200, "phone_linear_acceleration": 100,
    "phone_gyroscope_uncalibrated": 500, "phone_gyroscope": 500,
    "phone_magnetometer_uncalibrated": 100, "phone_magnetometer": 100,
    "phone_interrupt_gyroscope": 100, "watch_accelerometer": 100,
    "watch_gyroscope_uncalibrated": 200, "watch_linear_acceleration": 100,
    "watch_gyroscope": 100, "watch_magnetometer_uncalibrated": 100,
    "watch_magnetometer": 100, "watch_gravity": 100,
}
REFERENCE_KEYWORDS = ("accelerometer", "gyroscope", "magnetometer")
ALLOWED_ACTIVITIES = {
    "quick_walk","jogging","laying","reading","sitting","slow_walk","standing",
    "talk_using_phone","typing","walking","clean_the_table"
}
TARGET_MS, MIN_MS, AVG_DELTA_N = 180_000, 10_000, 5
REPORT_FILE = "report.csv"


In [58]:
@dataclass
class ReportRow:
    file_path: str
    sensor: str
    original_rows: int
    final_rows: int
    orig_start: int
    orig_end: int
    final_start: int
    final_end: int
    action: str
    warnings: str
    dropped_rows: int
    nan_rows: int
    mismatch_rate: float


In [60]:
def read_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, header=None, usecols=[0,1,2,3],
                     names=["t","x","y","z"], na_values=["","NA","nan"])
    nan_rows = df.isna().any(axis=1).sum()
    df = df.dropna(how="all")
    # timestamp normalization
    df["t"] = pd.to_numeric(df["t"], errors="coerce") * (1000 if df["t"].max()<1e11 else 1)
    df = df.dropna(subset=["t"])
    df = df.astype({"t":"int64"})
    df[["x","y","z"]] = df[["x","y","z"]].apply(pd.to_numeric, errors="coerce")
    dropped_rows = len(df) - df.dropna().shape[0]
    return df.dropna().reset_index(drop=True), dropped_rows, nan_rows

def ensure_monotonic(df: pd.DataFrame) -> pd.DataFrame:
    diffs = df["t"].diff().dropna()
    mismatch_rate = (diffs <= 0).mean() if not diffs.empty else 0.0
    eps = max(1, int(diffs.tail(AVG_DELTA_N).mean()/10)) if not diffs.empty else 1
    df = df.sort_values("t").reset_index(drop=True)
    for i in range(1,len(df)):
        if df.at[i,"t"] <= df.at[i-1,"t"]:
            df.at[i,"t"] = df.at[i-1,"t"] + eps
    return df, mismatch_rate


In [62]:
def extend_to_target(df: pd.DataFrame) -> pd.DataFrame:
    base, avg = df["t"].iloc[0], int(df["t"].diff().mean())
    while df["t"].iloc[-1] - base < TARGET_MS:
        chunk = df.copy()
        chunk["t"] += df["t"].iloc[-1] + avg - chunk["t"].iloc[0]
        df = pd.concat([df, chunk])
    return df[df["t"] <= base+TARGET_MS].reset_index(drop=True)

def finalize(df: pd.DataFrame) -> pd.DataFrame:
    base, desired = df["t"].iloc[0], df["t"].iloc[0]+TARGET_MS
    if df["t"].iloc[-1] < desired:
        row = df.iloc[[-1]].copy()
        row["t"] = desired
        df = pd.concat([df, row])
    df.at[len(df)-1,"t"] = desired
    return df


In [64]:
def process_file(path: Path, sensor: str, dry: bool) -> ReportRow:
    try:
        df, dropped, nan_rows = read_csv(path)
    except Exception as e:
        return ReportRow(str(path), sensor,0,0,0,0,0,0,"read_failed", str(e),0,0,0.0)
    if df.empty:
        return ReportRow(str(path), sensor,0,0,0,0,0,0,"empty","",0,0,0.0)

    orig_rows, start, end = len(df), df["t"].iloc[0], df["t"].iloc[-1]
    dur = end-start
    if dur < MIN_MS:
        return ReportRow(str(path), sensor,orig_rows,0,start,end,0,0,"too_short","",dropped,nan_rows,0.0)
    if dur > TARGET_MS:
        df = df[df["t"] <= start+TARGET_MS]

    df, mismatch_rate = ensure_monotonic(df)
    if dur < TARGET_MS:
        df = extend_to_target(df)
    df = finalize(df)

    final_rows = len(df)
    warnings = []
    if final_rows < orig_rows:
        warnings.append("row_reduction")

    if not dry:
        df.to_csv(path, index=False, header=False)

    return ReportRow(
        str(path), sensor, orig_rows, final_rows, start, end,
        int(df["t"].iloc[0]), int(df["t"].iloc[-1]),
        "processed", ";".join(warnings), dropped, nan_rows, mismatch_rate
    )


In [66]:
def process_all(base: Path, dry=False):
    reports = []
    for folder in base.rglob("*"):
        if not folder.is_dir() or folder.name not in ALLOWED_ACTIVITIES:
            continue
        for sensor in SENSOR_RATES:
            matches = [f for f in folder.glob("*.csv") if sensor in f.name.lower()]
            if not matches: 
                continue
            reports.append(process_file(matches[0], sensor, dry))
    df_report = pd.DataFrame([asdict(r) for r in reports])
    out = base / REPORT_FILE
    df_report.to_csv(out, index=False)
    return df_report

In [68]:
# Example usage (dry-run mode)
base_path = Path(r"C:\Users\Malik Haider\Documents\HUMCARE\DATASET_FILLING\test_data")
report = process_all(base_path, dry=False)

In [72]:
report.head()

,file_path,sensor,original_rows,final_rows,orig_start,orig_end,final_start,final_end,action,warnings,dropped_rows,nan_rows,mismatch_rate
0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_accelerometer,435,899,1750134569750,1750134656873,1750134569750,1750134749750,processed,,0,0,0.000000
1,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_gyroscope,420,899,1750134569750,1750134653858,1750134569750,1750134749750,processed,,0,0,0.000000
2,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_magnetometer,419,899,1750134569750,1750134653678,1750134569750,1750134749750,processed,,0,0,0.000000
3,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,phone_accelerometer,89757,89942,1750134569636,1750134749271,1750134569636,1750134749636,processed,,0,0,0.134465
4,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,phone_gravity,17731,17994,1750134569635,1750134747022,1750134569635,1750134749635,processed,,0,0,0.000056


In [2]:

import os
import re
import glob
import math
import argparse
import logging
from pathlib import Path
from typing import Optional, Tuple, List, Dict, Any

import numpy as np
import pandas as pd

SENSOR_RATES = {
    "glass_accelerometer": 5, "glass_gyroscope": 5, "glass_magnetometer": 5,
    "phone_acceleromter_calibrated": 100, "phone_accelerometer": 500,
    "phone_gravity": 200, "phone_linear_acceleration": 100,
    "phone_gyroscope_uncalibrated": 500, "phone_gyroscope": 500,
    "phone_magnetometer_uncalibrated": 100, "phone_magnetometer": 100,
    "phone_interrupt_gyroscope": 100, "watch_accelerometer": 100,
    "watch_gyroscope_uncalibrated": 200, "watch_linear_acceleration": 100,
    "watch_gyroscope": 100, "watch_magnetometer_uncalibrated": 100,
    "watch_magnetometer": 100, "watch_gravity": 100,
}

TOTAL_CHUNKS = 36
CHUNK_MS = 5_000
CHUNK_DURATION_S = CHUNK_MS / 1000.0
TARGET_MS = TOTAL_CHUNKS * CHUNK_MS  # expected original length: 180_000 ms (180s)
REPORT_FILENAME = "report.csv"

# Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")


# ---------------------------
# Utilities
# ---------------------------

def guess_sensor_from_filename(fname: str, sensors_map: Dict[str, int]) -> Optional[str]:
    """
    Return the best matching sensor key if any substring matches.
    """
    lower = fname.lower()
    for key in sensors_map:
        if key in lower:
            return key
    # also try replacing '-' or spaces with underscores
    lower_norm = lower.replace("-", "_").replace(" ", "_")
    for key in sensors_map:
        if key in lower_norm:
            return key
    return None


def read_sensor_csv(path: Path) -> Tuple[pd.DataFrame, int, int]:
    """
    Read CSV robustly. Returns (df, dropped_rows_count, nan_rows_count).
    Assumes first 4 columns are t,x,y,z (no header usually).
    Converts timestamps if necessary (seconds -> ms).
    Drops rows that have no timestamp.
    """
    # read first 4 columns, tolerant to different formats
    try:
        df = pd.read_csv(path, header=None, usecols=[0, 1, 2, 3], names=["t", "x", "y", "z"],
                         dtype={0: object, 1: object, 2: object, 3: object}, na_values=["", "NA", "nan"])
    except Exception as e:
        # fallback: try without usecols (some files may have different column count)
        df = pd.read_csv(path, header=None, names=["t", "x", "y", "z"], na_values=["", "NA", "nan"])
        df = df.iloc[:, :4]
        df.columns = ["t", "x", "y", "z"]

    # count rows with any NaN (before dropping)
    nan_rows = df.isna().any(axis=1).sum()

    # drop rows where all are NaN
    df = df.dropna(how="all").copy()

    # normalize timestamp column: numeric coercion
    df["t"] = pd.to_numeric(df["t"], errors="coerce")

    # convert seconds -> milliseconds if the timestamps look like seconds
    # typical epoch in ms is > 1e11; if max timestamp is < 1e11 assume seconds
    if not df["t"].empty:
        max_t = df["t"].max()
        if max_t is not None and max_t < 1e11:
            df["t"] = df["t"] * 1000.0

    df = df.dropna(subset=["t"]).copy()
    df["t"] = df["t"].astype("int64")
    # ensure numeric x,y,z
    df[["x", "y", "z"]] = df[["x", "y", "z"]].apply(pd.to_numeric, errors="coerce")
    # how many rows had numeric problems (x,y,z) - we keep them (they may be NaN) but count dropped later if needed
    dropped_rows = 0  # we don't drop partially numeric rows here
    df = df.reset_index(drop=True)
    return df, dropped_rows, int(nan_rows)


def estimate_rate_from_df(df: pd.DataFrame, fallback: int = 50) -> int:
    """
    Estimate sampling rate (Hz) from median delta of timestamps.
    """
    if len(df) < 2:
        return fallback
    deltas = df["t"].diff().dropna()
    # use median to be robust
    median_ms = deltas.median()
    if pd.isna(median_ms) or median_ms <= 0:
        return fallback
    est_rate = int(round(1000.0 / median_ms))
    return max(1, est_rate)


def ensure_monotonic_timestamps(df: pd.DataFrame, min_step_ms: int = 1) -> Tuple[pd.DataFrame, float]:
    """
    Sort by timestamp and make strictly increasing by enforcing minimum step (min_step_ms).
    Returns corrected df and mismatch_rate (fraction of non-increasing deltas in the original).
    """
    if df.empty or len(df) < 2:
        return df, 0.0
    # compute mismatch rate before sort
    diffs = df["t"].diff().dropna()
    mismatch_rate = float((diffs <= 0).mean()) if not diffs.empty else 0.0

    # sort
    df = df.sort_values("t").reset_index(drop=True)
    # repair equal or decreasing timestamps
    for i in range(1, len(df)):
        if df.at[i, "t"] <= df.at[i-1, "t"]:
            df.at[i, "t"] = df.at[i-1, "t"] + min_step_ms
    return df, mismatch_rate


# ---------------------------
# Core splitting logic
# ---------------------------

def create_chunk_rows(original_df: pd.DataFrame, window_start: int, window_end: int, target_rows: int) -> pd.DataFrame:
    """
    Build a chunk dataframe of exactly target_rows rows for the time window [window_start, window_end).
    Padding: copy from start of the original file (cycling if necessary).
    After assembling x,y,z values, timestamps are overwritten with evenly spaced ints over [window_start, window_end-1].
    The returned df has columns ['t','x','y','z'].
    """
    # select rows that fall into the window
    mask = (original_df["t"] >= window_start) & (original_df["t"] < window_end)
    chunk = original_df.loc[mask, ["x", "y", "z"]].copy().reset_index(drop=True)

    available = len(chunk)
    needed = target_rows - available

    if needed > 0:
        # pad from the start of the file as required, cycling if needed
        # take values only (x,y,z) from the original df start
        source = original_df[["x", "y", "z"]].reset_index(drop=True)
        if source.empty:
            # Create zeros if file is empty - improbable but safe fallback
            pad_vals = pd.DataFrame(np.zeros((needed, 3)), columns=["x", "y", "z"])
        else:
            # repeat source rows until we have enough
            reps = int(math.ceil(needed / len(source)))
            padded_pool = pd.concat([source] * reps, ignore_index=True)
            pad_vals = padded_pool.iloc[:needed].reset_index(drop=True)
        chunk = pd.concat([chunk, pad_vals], ignore_index=True)

    # trim extra just in case
    if len(chunk) > target_rows:
        chunk = chunk.iloc[:target_rows].reset_index(drop=True)

    # assign evenly spaced timestamps across [window_start, window_end-1]
    ts = np.linspace(window_start, window_end - 1, target_rows)
    ts = np.round(ts).astype("int64")
    chunk.insert(0, "t", ts)
    return chunk


def split_file_into_chunks(path: Path, rate_hz: int, total_chunks: int = TOTAL_CHUNKS, chunk_ms: int = CHUNK_MS) -> Tuple[List[Path], List[Dict[str, Any]]]:
    """
    Split the 180s file into `total_chunks` files. Returns list of output paths and a list of metadata dicts
    (one dict per chunk) for reporting.
    """
    df, dropped_rows, nan_rows = read_sensor_csv(path)
    if df.empty:
        raise ValueError(f"File {path} has no valid rows after reading.")

    # Ensure timestamps are monotonically increasing
    df, mismatch_rate = ensure_monotonic_timestamps(df)

    start_ts = int(df["t"].iloc[0])
    target_rows = int(rate_hz * (chunk_ms / 1000.0))

    out_paths = []
    report_rows = []

    # ensure the original covers at least 180s? If shorter, we will cycle/pad across windows
    for i in range(total_chunks):
        chunk_start = start_ts + i * chunk_ms
        chunk_end = chunk_start + chunk_ms
        chunk_df = create_chunk_rows(df, chunk_start, chunk_end, target_rows)

        # final safety: ensure exactly target_rows and timestamps strictly increasing
        if len(chunk_df) != target_rows:
            chunk_df = chunk_df.iloc[:target_rows].reset_index(drop=True)

        chunk_df, _ = ensure_monotonic_timestamps(chunk_df, min_step_ms=max(1, int(chunk_df["t"].diff().median() or 1)))
        out_name = f"{path.stem}_e{i+1}.csv"
        out_path = path.with_name(out_name)
        # write
        chunk_df.to_csv(out_path, index=False, header=False)
        out_paths.append(out_path)

        # record report row
        report_rows.append({
            "orig_file": str(path),
            "sensor": path.stem,
            "rate_hz": rate_hz,
            "chunk_index": i + 1,
            "chunk_start": chunk_start,
            "chunk_end": chunk_end,
            "rows_written": len(chunk_df),
            "padded": int(len(chunk_df) != ((df["t"] >= chunk_start) & (df["t"] < chunk_end)).sum()),
            "out_path": str(out_path),
            "dropped_rows_in_read": dropped_rows,
            "nan_rows_in_read": nan_rows,
            "mismatch_rate_before_fix": mismatch_rate,
        })

    return out_paths, report_rows


# ---------------------------
# High level processing
# ---------------------------

def process_file(path: Path, sensor_rates_map: Dict[str, int], remove_original: bool = False, dry_run: bool = True) -> List[Dict[str, Any]]:
    """
    Process a single file. Returns list of per-chunk report dicts.
    """
    fname = path.name
    # skip already-split files
    if re.search(r"_e\d+\.csv$", fname):
        logging.debug(f"Skipping already-chunked file: {path}")
        return []

    # try to guess sensor type from filename
    sensor_key = guess_sensor_from_filename(fname, sensor_rates_map)
    # fallback - try to use stem exact match
    if sensor_key is None:
        sensor_key = guess_sensor_from_filename(path.stem, sensor_rates_map)

    # read to estimate rate if needed
    df, dropped_rows, nan_rows = read_sensor_csv(path)
    if df.empty:
        logging.warning(f"Skipping empty/invalid file: {path}")
        return [{
            "orig_file": str(path),
            "error": "empty_or_invalid_file",
            "dropped_rows": dropped_rows,
            "nan_rows": nan_rows
        }]

    # choose rate
    if sensor_key:
        rate = sensor_rates_map[sensor_key]
        logging.info(f"Using mapped rate {rate}Hz for {sensor_key} (file: {fname})")
    else:
        rate = estimate_rate_from_df(df, fallback=50)
        logging.info(f"No mapping for '{fname}' — estimated rate {rate}Hz")

    # if the original file is shorter/longer than expected, we won't try to trim here; splitting windows are anchored to start timestamp
    # call split
    try:
        # we pass the already-read df by writing it temporarily to ensure single read? To keep code simple we call split_file_into_chunks which reads again.
        out_paths, report_rows = split_file_into_chunks(path, rate, total_chunks=TOTAL_CHUNKS, chunk_ms=CHUNK_MS)
        # optionally remove original
        if remove_original and not dry_run:
            try:
                path.unlink()
                logging.info(f"Removed original file: {path}")
            except Exception as e:
                logging.warning(f"Could not remove original {path}: {e}")
        # annotate reports with orig file-level fields
        for r in report_rows:
            r["orig_rows"] = len(df)
            r["orig_start"] = int(df["t"].iloc[0])
            r["orig_end"] = int(df["t"].iloc[-1])
        return report_rows
    except Exception as e:
        logging.exception(f"Failed to split file {path}: {e}")
        return [{
            "orig_file": str(path),
            "error": str(e)
        }]


def process_all(base: Path, sensor_rates_map: Dict[str, int], remove_original: bool = False, dry_run: bool = True, recurse: bool = True):
    """
    Walk the base path (files if file provided or folder recursively) and process sensor CSVs.
    Produces a report CSV at base/REPORT_FILENAME (if base is a directory) or in the file's parent.
    Returns pandas DataFrame of report.
    """
    base = Path(base)
    report_rows: List[Dict[str, Any]] = []

    if base.is_file():
        report_rows.extend(process_file(base, sensor_rates_map, remove_original=remove_original, dry_run=dry_run))
        out_dir = base.parent
    else:
        # recurse search for CSVs
        pattern = "**/*.csv" if recurse else "*.csv"
        files = list(base.glob(pattern))
        files = [p for p in files if not re.search(r"_e\d+\.csv$", p.name)]  # skip already chunked
        logging.info(f"Found {len(files)} csv files under {base}")
        for f in files:
            try:
                rows = process_file(f, sensor_rates_map, remove_original=remove_original, dry_run=dry_run)
                report_rows.extend(rows)
            except Exception as e:
                logging.exception(f"Error processing {f}: {e}")
        out_dir = base

    if report_rows:
        df_report = pd.DataFrame(report_rows)
    else:
        df_report = pd.DataFrame(columns=[
            "orig_file", "sensor", "rate_hz", "chunk_index", "chunk_start", "chunk_end",
            "rows_written", "padded", "out_path", "dropped_rows_in_read", "nan_rows_in_read",
            "mismatch_rate_before_fix", "orig_rows", "orig_start", "orig_end"
        ])
    # write out CSV
    report_path = out_dir / REPORT_FILENAME
    if not dry_run:
        df_report.to_csv(report_path, index=False)
        logging.info(f"Wrote report to {report_path}")
    else:
        logging.info("Dry run enabled — not writing report file to disk.")

    return df_report


In [4]:
from pathlib import Path
base_path = Path(r"C:\Users\Malik Haider\Documents\HUMCARE\DATASET_FILLING\test_data")
report = process_all(base_path, SENSOR_RATES, remove_original=True, dry_run=False)

2025-09-19 23:42:32,090 INFO: Found 602 csv files under C:\Users\Malik Haider\Documents\HUMCARE\DATASET_FILLING\test_data
2025-09-19 23:42:32,187 INFO: Using mapped rate 5Hz for glass_accelerometer (file: glass_accelerometer.csv)
2025-09-19 23:42:32,720 INFO: Removed original file: C:\Users\Malik Haider\Documents\HUMCARE\DATASET_FILLING\test_data\areeba\clean_the_table\glass_accelerometer.csv
2025-09-19 23:42:32,787 INFO: Using mapped rate 5Hz for glass_gyroscope (file: glass_gyroscope.csv)
2025-09-19 23:42:33,281 INFO: Removed original file: C:\Users\Malik Haider\Documents\HUMCARE\DATASET_FILLING\test_data\areeba\clean_the_table\glass_gyroscope.csv
2025-09-19 23:42:33,346 INFO: Using mapped rate 5Hz for glass_magnetometer (file: glass_magnetometer.csv)
2025-09-19 23:42:33,827 INFO: Removed original file: C:\Users\Malik Haider\Documents\HUMCARE\DATASET_FILLING\test_data\areeba\clean_the_table\glass_magnetometer.csv
2025-09-19 23:42:34,489 INFO: Using mapped rate 500Hz for phone_acceler

In [10]:
report.head(100)

,orig_file,sensor,rate_hz,chunk_index,chunk_start,chunk_end,rows_written,padded,out_path,dropped_rows_in_read,nan_rows_in_read,mismatch_rate_before_fix,orig_rows,orig_start,orig_end,error,dropped_rows,nan_rows
0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_accelerometer,5.0,1.0,1.750135e+12,1.750135e+12,25.0,1.0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,0.0,0.0,0.0,435.0,1.750135e+12,1.750135e+12,NaN,NaN,NaN
1,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_accelerometer,5.0,2.0,1.750135e+12,1.750135e+12,25.0,1.0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,0.0,0.0,0.0,435.0,1.750135e+12,1.750135e+12,NaN,NaN,NaN
2,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_accelerometer,5.0,3.0,1.750135e+12,1.750135e+12,25.0,0.0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,0.0,0.0,0.0,435.0,1.750135e+12,1.750135e+12,NaN,NaN,NaN
3,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_accelerometer,5.0,4.0,1.750135e+12,1.750135e+12,25.0,0.0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,0.0,0.0,0.0,435.0,1.750135e+12,1.750135e+12,NaN,NaN,NaN
4,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_accelerometer,5.0,5.0,1.750135e+12,1.750135e+12,25.0,0.0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,0.0,0.0,0.0,435.0,1.750135e+12,1.750135e+12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_magnetometer,5.0,24.0,1.750135e+12,1.750135e+12,25.0,1.0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,0.0,0.0,0.0,419.0,1.750135e+12,1.750135e+12,NaN,NaN,NaN
96,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_magnetometer,5.0,25.0,1.750135e+12,1.750135e+12,25.0,1.0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,0.0,0.0,0.0,419.0,1.750135e+12,1.750135e+12,NaN,NaN,NaN
97,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_magnetometer,5.0,26.0,1.750135e+12,1.750135e+12,25.0,1.0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,0.0,0.0,0.0,419.0,1.750135e+12,1.750135e+12,NaN,NaN,NaN
98,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,glass_magnetometer,5.0,27.0,1.750135e+12,1.750135e+12,25.0,1.0,C:\Users\Malik Haider\Documents\HUMCARE\DATASE...,0.0,0.0,0.0,419.0,1.750135e+12,1.750135e+12,NaN,NaN,NaN
